## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

在機器學習中，我們通常將原始數據按照比例分割為“測試集”和“訓練集”，通常使用sklearn.cross_validation裡的train_test_split模塊用來分割數據。

## 語法架構
X_train,X_test, y_train, y_test =sklearn.model_selection.train_test_split(train_data,train_target,test_size=0.4, random_state=0, stratify= y_train )

### 參數說明
- train_data：所要劃分的樣本特徵集。
- train_target：所要劃分的樣本結果。
- test_size：樣本佔比，如果是整數的話就是樣本的數量。
- random_state：是隨機數的種子。(隨機數種子：其實就是該組隨機數的編號，在需要重複試驗的時候，保證得到一組一樣的隨機數。比如你每次都填1，其他參數一樣的情況下你得到的隨機數組是一樣的。但填0或不填，每次都會不一樣。)
****************

In [2]:
# 讀取手寫辨識資料集
wine = datasets.load_wine()
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 0.14672 的 MSE
# 訓練模型
clf.fit(x_train, y_train)
# 預測測試集
y_pred = clf.predict(x_test)
# 查看MSE
print(metrics.mean_squared_error(y_test, y_pred))

0.1436066329824748


----
### GridSearchCV 超參數說明

- n_estimators = number of trees in the foreset (森林中樹木的數量)

- max_features = max number of features considered for splitting a node
- max_depth = max number of levels in each decision tree (樹的最大深度)

- min_samples_split = min number of data points placed in a node before the node is split (拆分內部節點所需的最少樣本數)

- min_samples_leaf = min number of data points allowed in a leaf node (在葉節點處需要的最小樣本數)

- bootstrap = method for sampling data points (with or without replacement) (建立樹木時是否使用引導程序樣本)

In [4]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    2.5s finished


In [5]:
# 列印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.061972 using {'max_depth': 5, 'n_estimators': 100}


In [6]:
# 最佳參數
grid_result.best_params_

{'max_depth': 5, 'n_estimators': 100}

### 梯度提升機參數說明

clf = GradientBoostingClassifier(loss, max_features, learning_rate, n_estimators)

- loss="deviance": Loss 的選擇，若改為`exponential`則會變成Adaboosting 演算法，概念相同但實作稍微不同

- max_features="auto": 如何選取`features`

- learning_rate=0.1: 每棵樹對最終結果的影響，應與`n_estimators`成反比

- n_estimators=100: 決策樹的數量

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [8]:
# 調整參數後約可降至 0.030 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

0.18730001728275358
